<a href="https://colab.research.google.com/github/cleysonl/Tensorflow2.0/blob/master/ANN_and_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **ANN with TF2.0**

In [1]:
!pip install tensorflow-gpu==2.0.0.alpha0

     |████████████████████████████████| 332.1MB 44kB/s 
     |████████████████████████████████| 419kB 13.8MB/s 
     |████████████████████████████████| 3.0MB 52.7MB/s 


In [0]:
# Import libraries
import os
import tensorflow as tf
import numpy as np

In [0]:
#Dataset
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' # or any {'0',' 1', '2'}

def mnist_dataset():
  (x,y),_ = tf.keras.datasets.mnist.load_data()
  ds = tf.data.Dataset.from_tensor_slices((x,y))
  ds = ds.map(prepare_mnist_features_and_labels)
  ds = ds.take(20000).shuffle(20000).batch(100)
  return ds

def prepare_mnist_features_and_labels(x,y):
  x = tf.cast(x, tf.float32)/255.
  y = tf.cast(y, tf.int64)
  return x, y

In [0]:
#Define the model
model = tf.keras.models.Sequential()
# Define the layers
model.add(tf.keras.layers.Reshape(target_shape=(28*28,),input_shape=(28,28)))
model.add(tf.keras.layers.Dense(units=100, activation='relu'))
model.add(tf.keras.layers.Dense(units=100, activation='relu'))
model.add(tf.keras.layers.Dense(units=10))
#define the optimizer
optimizer = tf.keras.optimizers.Adam()

In [0]:
@tf.function
def compute_loss(logits, labels):
  return tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(
          logits=logits, labels=labels))

@tf.function
def compute_accuracy(logits, labels):
  predictions = tf.argmax(logits, axis=1)
  return tf.reduce_mean(tf.cast(tf.equal(predictions, labels), tf.float32)) 

@tf.function
def train_one_step(model, optimizer, x, y):

  with tf.GradientTape() as tape:
    logits = model(x)
    loss = compute_loss(logits, y)

    #compute gradient
    grads = tape.gradient(loss, model.trainable_variables)
    #update weights
    optimizer.apply_gradients(zip(grads, model.trainable_variables))

    accuracy = compute_accuracy(logits, y)

    #loss and accuracy is scalar tensor
    return loss, accuracy

def train(epoch, model, optimizer):
  train_ds = mnist_dataset()
  loss = 0.0
  accuracy = 0.0
  for step, (x,y) in enumerate(train_ds):
    loss, accuracy = train_one_step(model, optimizer, x, y)

    if step%500 == 0:
      print('epoch', epoch, ':loss', loss.numpy(), ': accuracy', accuracy.numpy())

  return loss, accuracy


In [0]:
for epoch in range(20):
  loss, accuracy = train(epoch, model, optimizer)

print('Final epoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())

epoch 0 :loss 2.3029244 : accuracy 0.13
epoch 1 :loss 0.16009605 : accuracy 0.95
epoch 2 :loss 0.17455128 : accuracy 0.95
epoch 3 :loss 0.086833164 : accuracy 0.98
epoch 4 :loss 0.09488362 : accuracy 0.96
epoch 5 :loss 0.037679218 : accuracy 1.0
epoch 6 :loss 0.049664084 : accuracy 0.98
epoch 7 :loss 0.029889045 : accuracy 1.0
epoch 8 :loss 0.008005618 : accuracy 1.0
epoch 9 :loss 0.028494503 : accuracy 0.99
epoch 10 :loss 0.013228629 : accuracy 1.0
epoch 11 :loss 0.01597538 : accuracy 1.0
epoch 12 :loss 0.00436632 : accuracy 1.0
epoch 13 :loss 0.0071314042 : accuracy 1.0
epoch 14 :loss 0.007745029 : accuracy 1.0
epoch 15 :loss 0.0046918783 : accuracy 1.0
epoch 16 :loss 0.004132434 : accuracy 1.0
epoch 17 :loss 0.0023726213 : accuracy 1.0
epoch 18 :loss 0.003863144 : accuracy 1.0
epoch 19 :loss 0.0038915456 : accuracy 1.0
Final epoch 19 : loss 0.0015582936 ; accuracy 1.0


#**CNN with TF2.0**

In [0]:
import os
import time
import numpy as np
import tensorflow as tf
from tensorflow.python.ops import summary_ops_v2
from tensorflow import keras
from tensorflow.keras import datasets, layers, models, optimizers, metrics

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'


In [4]:
def mnist_dataset():
  (X_train, y_train), (X_test, y_test) = datasets.mnist.load_data()

  X_train, X_test = X_train/np.float32(255), X_test/np.float32(255)
  y_train, y_test = y_train.astype(np.int64), y_test.astype(np.int64)

  train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
  test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))
  return train_dataset, test_dataset

train_ds, test_ds = mnist_dataset()
train_ds = train_ds.shuffle(60000).batch(100)
test_ds = test_ds.batch(100)

11493376/11490434 [==============================] - 0s 0us/step


In [0]:
model = tf.keras.Sequential()
model.add(layers.Reshape(target_shape=[28,28,1], input_shape=(28, 28,)))
model.add(layers.Conv2D(filters = 2, kernel_size = 5,padding='same', activation='relu'))
model.add(layers.MaxPooling2D(pool_size = (2,2), strides= (2,2), padding='same',))
model.add(layers.Conv2D(filters = 4, kernel_size = 5, padding ='same'))
model.add(layers.MaxPooling2D(pool_size = (2,2), strides= (2,2), padding='same',))
model.add(layers.Flatten())
model.add(layers.Dense(units = 32, activation='relu'))
model.add(layers.Dropout(rate=0.4))
model.add(layers.Dense(units=10))

compute_loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
compute_accuracy = tf.keras.metrics.SparseTopKCategoricalAccuracy()
optimizer = optimizers.SGD(learning_rate=0.01, momentum = 0.5)

In [0]:
def train_step(model, optimizer, images, labels):

  with tf.GradientTape() as tape:
    logits = model(images, training=True)
    loss = compute_loss(labels, logits)
    compute_accuracy(labels, logits)

  grads = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(grads, model.trainable_variables))
  
  return loss

  